# **Imports**

In [76]:
import snowflake.connector as snwconn
import pandas as pd
import os
from dotenv import load_dotenv

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

load_dotenv()

True

# **Snowflake Connections**

In [3]:
conn_params = {
    'account': os.getenv('SNW_ACCOUNT').replace('.', '-'),
    'user': os.getenv('SNW_USERNAME'),
    'password': os.getenv('SNW_PASS'),
    'warehouse': 'COMPUTE_WH',
    'database': 'JMAN_FINAL_PROJECT',
    'schema': 'DBO_STAGING'
}

In [4]:
conn = snwconn.connect(**conn_params)
cursor = conn.cursor()

# **Dataset Table**

## Read Dataset

In [5]:
csv_file_path = 'data/all_data.csv'
force_cloud = False

In [6]:
def _read_from_cloud():
    with open('amalgamation.sql', 'r') as sql_query_file:
        sql_query_text = sql_query_file.read()
    
    cursor.execute(sql_query_text)
    data = cursor.fetchall()
    columns = [col[0] for col in cursor.description]
    main_df = pd.DataFrame(data, columns=columns)

    return main_df

In [7]:
def _save_to_local(df): 
    dirs_to_make = '/'.join(csv_file_path.split('/')[:-1])
    if not os.path.exists(dirs_to_make):
        os.makedirs(dirs_to_make)

    df.to_csv(csv_file_path, index=False)

In [8]:
if not force_cloud and os.path.exists(csv_file_path):
    print('CSV Exists, reading from local')
    main_df = pd.read_csv(csv_file_path)
else:
    print('CSV does\'t Exist, reading from cloud')
    main_df = _read_from_cloud()
    _save_to_local(main_df)

CSV Exists, reading from local


## Dataset Overview

In [9]:
main_df

,USER_EMAIL,USER_ROLE,PROJECT_NAME,PROJECT_DOMAIN,DATE_START,DATE_END,TIMESHEET_COMMENT_ADDED,TIMESHEET_D0,TIMESHEET_D1,TIMESHEET_D2,TIMESHEET_D3,TIMESHEET_D4,TIMESHEET_D5,TIMESHEET_D6,TIMESHEET_ACTIVITY_NAME,FEEDBACK_QUESTION,FEEDBACK_CHECKED_ANSWER,FEEDBACK_TEXT_ANSWER
0,wbriggs@example.net,user,Chaney Inc Solution,Media and Entertainment,04-09-2022,10-09-2022,NaN,0,1,1,2,0,2,1,Sales Activity,NaN,NaN,NaN
1,wbriggs@example.net,user,"Wilson, Jackson and Green Solution",Education Technology,04-09-2022,10-09-2022,Excited to get started!,1,1,0,1,1,0,0,BAU Activity,NaN,NaN,NaN
2,wbriggs@example.net,user,"Wilson, Jackson and Green Solution",Education Technology,04-09-2022,10-09-2022,Excited to get started!,1,1,0,1,1,0,0,Sales Activity,NaN,NaN,NaN
3,wbriggs@example.net,user,"Gonzales, Martin and Carter Initiative",Media and Entertainment,04-09-2022,10-09-2022,Excited to get started!,0,1,2,0,2,2,0,Sales Activity,NaN,NaN,NaN
4,wbriggs@example.net,user,"Gonzales, Martin and Carter Initiative",Media and Entertainment,04-09-2022,10-09-2022,Excited to get started!,0,1,2,0,2,2,0,BAU Activity,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171598,rodneyfischer@example.org,admin,Olson-Craig Solution,Cybersecurity,25-12-2022,31-12-2022,Ready to dive in,2,2,8,1,7,0,0,Sales Activity,Were the project goals clearly defined? Please...,5.0,NaN
171599,rodneyfischer@example.org,admin,Olson-Craig Solution,Cybersecurity,25-12-2022,31-12-2022,Ready to dive in,2,2,8,1,7,0,0,BAU Activity,Were the project goals clearly defined? Please...,5.0,NaN
171600,rodneyfischer@example.org,admin,Saunders LLC Solution,Healthcare,10-12-2023,16-12-2023,NaN,0,1,9,0,6,4,1,Sales Activity,NaN,NaN,NaN
171601,rodneyfischer@example.org,admin,Briggs-Jennings Project,Healthcare,22-05-2022,28-05-2022,NaN,1,3,0,0,0,10,0,BAU Activity,NaN,NaN,NaN


In [10]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171603 entries, 0 to 171602
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   USER_EMAIL               171603 non-null  object 
 1   USER_ROLE                171603 non-null  object 
 2   PROJECT_NAME             171603 non-null  object 
 3   PROJECT_DOMAIN           171603 non-null  object 
 4   DATE_START               171603 non-null  object 
 5   DATE_END                 171603 non-null  object 
 6   TIMESHEET_COMMENT_ADDED  85913 non-null   object 
 7   TIMESHEET_D0             171603 non-null  int64  
 8   TIMESHEET_D1             171603 non-null  int64  
 9   TIMESHEET_D2             171603 non-null  int64  
 10  TIMESHEET_D3             171603 non-null  int64  
 11  TIMESHEET_D4             171603 non-null  int64  
 12  TIMESHEET_D5             171603 non-null  int64  
 13  TIMESHEET_D6             171603 non-null  int64  
 14  TIME

## Fill Nulls from dataset

In [11]:
main_df.isnull().sum(axis=0)

USER_EMAIL                      0
USER_ROLE                       0
PROJECT_NAME                    0
PROJECT_DOMAIN                  0
DATE_START                      0
DATE_END                        0
TIMESHEET_COMMENT_ADDED     85690
TIMESHEET_D0                    0
TIMESHEET_D1                    0
TIMESHEET_D2                    0
TIMESHEET_D3                    0
TIMESHEET_D4                    0
TIMESHEET_D5                    0
TIMESHEET_D6                    0
TIMESHEET_ACTIVITY_NAME         0
FEEDBACK_QUESTION          158216
FEEDBACK_CHECKED_ANSWER    158216
FEEDBACK_TEXT_ANSWER       168818
dtype: int64

In [12]:
main_df = main_df.fillna('')

In [13]:
main_df.isnull().sum(axis=0)

USER_EMAIL                 0
USER_ROLE                  0
PROJECT_NAME               0
PROJECT_DOMAIN             0
DATE_START                 0
DATE_END                   0
TIMESHEET_COMMENT_ADDED    0
TIMESHEET_D0               0
TIMESHEET_D1               0
TIMESHEET_D2               0
TIMESHEET_D3               0
TIMESHEET_D4               0
TIMESHEET_D5               0
TIMESHEET_D6               0
TIMESHEET_ACTIVITY_NAME    0
FEEDBACK_QUESTION          0
FEEDBACK_CHECKED_ANSWER    0
FEEDBACK_TEXT_ANSWER       0
dtype: int64

## Cleaning

In [14]:
main_df['DATE_START'] = pd.to_datetime(main_df['DATE_START'], format='%d-%m-%Y')
main_df['DATE_END'] = pd.to_datetime(main_df['DATE_END'], format='%d-%m-%Y')

In [15]:
main_df = main_df.drop_duplicates()

In [16]:
main_df

,USER_EMAIL,USER_ROLE,PROJECT_NAME,PROJECT_DOMAIN,DATE_START,DATE_END,TIMESHEET_COMMENT_ADDED,TIMESHEET_D0,TIMESHEET_D1,TIMESHEET_D2,TIMESHEET_D3,TIMESHEET_D4,TIMESHEET_D5,TIMESHEET_D6,TIMESHEET_ACTIVITY_NAME,FEEDBACK_QUESTION,FEEDBACK_CHECKED_ANSWER,FEEDBACK_TEXT_ANSWER
0,wbriggs@example.net,user,Chaney Inc Solution,Media and Entertainment,2022-09-04,2022-09-10,,0,1,1,2,0,2,1,Sales Activity,,,
1,wbriggs@example.net,user,"Wilson, Jackson and Green Solution",Education Technology,2022-09-04,2022-09-10,Excited to get started!,1,1,0,1,1,0,0,BAU Activity,,,
2,wbriggs@example.net,user,"Wilson, Jackson and Green Solution",Education Technology,2022-09-04,2022-09-10,Excited to get started!,1,1,0,1,1,0,0,Sales Activity,,,
3,wbriggs@example.net,user,"Gonzales, Martin and Carter Initiative",Media and Entertainment,2022-09-04,2022-09-10,Excited to get started!,0,1,2,0,2,2,0,Sales Activity,,,
4,wbriggs@example.net,user,"Gonzales, Martin and Carter Initiative",Media and Entertainment,2022-09-04,2022-09-10,Excited to get started!,0,1,2,0,2,2,0,BAU Activity,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171598,rodneyfischer@example.org,admin,Olson-Craig Solution,Cybersecurity,2022-12-25,2022-12-31,Ready to dive in,2,2,8,1,7,0,0,Sales Activity,Were the project goals clearly defined? Please...,5.0,
171599,rodneyfischer@example.org,admin,Olson-Craig Solution,Cybersecurity,2022-12-25,2022-12-31,Ready to dive in,2,2,8,1,7,0,0,BAU Activity,Were the project goals clearly defined? Please...,5.0,
171600,rodneyfischer@example.org,admin,Saunders LLC Solution,Healthcare,2023-12-10,2023-12-16,,0,1,9,0,6,4,1,Sales Activity,,,
171601,rodneyfischer@example.org,admin,Briggs-Jennings Project,Healthcare,2022-05-22,2022-05-28,,1,3,0,0,0,10,0,BAU Activity,,,


# **Select Features**

In [17]:
attendance_regulariation_columns = ['USER_EMAIL', 'USER_ROLE', 'PROJECT_NAME', 'PROJECT_DOMAIN', 'TIMESHEET_D0', 'TIMESHEET_D1', 'TIMESHEET_D2',
                    'TIMESHEET_D3', 'TIMESHEET_D4', 'TIMESHEET_D5', 'TIMESHEET_D6', 'DATE_START',
                    'DATE_END', 'TIMESHEET_ACTIVITY_NAME']

attendance_regulariation_df = main_df[attendance_regulariation_columns].copy()
attendance_regulariation_df.drop_duplicates()
attendance_regulariation_df

,USER_EMAIL,USER_ROLE,PROJECT_NAME,PROJECT_DOMAIN,TIMESHEET_D0,TIMESHEET_D1,TIMESHEET_D2,TIMESHEET_D3,TIMESHEET_D4,TIMESHEET_D5,TIMESHEET_D6,DATE_START,DATE_END,TIMESHEET_ACTIVITY_NAME
0,wbriggs@example.net,user,Chaney Inc Solution,Media and Entertainment,0,1,1,2,0,2,1,2022-09-04,2022-09-10,Sales Activity
1,wbriggs@example.net,user,"Wilson, Jackson and Green Solution",Education Technology,1,1,0,1,1,0,0,2022-09-04,2022-09-10,BAU Activity
2,wbriggs@example.net,user,"Wilson, Jackson and Green Solution",Education Technology,1,1,0,1,1,0,0,2022-09-04,2022-09-10,Sales Activity
3,wbriggs@example.net,user,"Gonzales, Martin and Carter Initiative",Media and Entertainment,0,1,2,0,2,2,0,2022-09-04,2022-09-10,Sales Activity
4,wbriggs@example.net,user,"Gonzales, Martin and Carter Initiative",Media and Entertainment,0,1,2,0,2,2,0,2022-09-04,2022-09-10,BAU Activity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171598,rodneyfischer@example.org,admin,Olson-Craig Solution,Cybersecurity,2,2,8,1,7,0,0,2022-12-25,2022-12-31,Sales Activity
171599,rodneyfischer@example.org,admin,Olson-Craig Solution,Cybersecurity,2,2,8,1,7,0,0,2022-12-25,2022-12-31,BAU Activity
171600,rodneyfischer@example.org,admin,Saunders LLC Solution,Healthcare,0,1,9,0,6,4,1,2023-12-10,2023-12-16,Sales Activity
171601,rodneyfischer@example.org,admin,Briggs-Jennings Project,Healthcare,1,3,0,0,0,10,0,2022-05-22,2022-05-28,BAU Activity


# **Feature Engineering**

## Extracting required Columns and adding more Columns

In [18]:
# Group by USER_EMAIL and PROJECT_DATE_START, then calculate the sum of total hours worked
attendance_regulariation_df = attendance_regulariation_df.groupby(['USER_EMAIL', 'DATE_START'])[['TIMESHEET_D0', 'TIMESHEET_D1', 
                                                                      'TIMESHEET_D2', 'TIMESHEET_D3', 
                                                                      'TIMESHEET_D4', 'TIMESHEET_D5', 
                                                                      'TIMESHEET_D6']].sum()

attendance_regulariation_df['TOTAL_HOURS_WORKED_FOR_WEEK'] = attendance_regulariation_df.sum(axis=1)
attendance_regulariation_df['AVERAGE_DAILY_HOURS_WORKED'] = attendance_regulariation_df.mean(axis=1)
attendance_regulariation_df.reset_index(inplace=True)
attendance_regulariation_df

,USER_EMAIL,DATE_START,TIMESHEET_D0,TIMESHEET_D1,TIMESHEET_D2,TIMESHEET_D3,TIMESHEET_D4,TIMESHEET_D5,TIMESHEET_D6,TOTAL_HOURS_WORKED_FOR_WEEK,AVERAGE_DAILY_HOURS_WORKED
0,aaron25@example.com,2022-07-24,5,2,11,10,2,5,3,38,9.50
1,aaron25@example.com,2022-07-31,6,12,12,7,4,8,3,52,13.00
2,aaron25@example.com,2022-10-16,0,16,2,6,2,0,8,34,8.50
3,aaron25@example.com,2023-03-05,7,2,9,3,7,9,5,42,10.50
4,aaron25@example.com,2023-05-21,2,0,0,1,5,3,0,11,2.75
...,...,...,...,...,...,...,...,...,...,...,...
45668,zwilliamson@example.org,2023-12-17,3,5,6,8,8,8,2,40,10.00
45669,zwilliamson@example.org,2023-12-24,4,14,4,8,4,12,12,58,14.50
45670,zwilliamson@example.org,2024-02-25,4,6,10,8,6,10,8,52,13.00
45671,zwolf@example.org,2022-06-26,0,6,0,1,2,0,0,9,2.25


In [19]:
weekly_regularization_threshold = 40
daily_regularization_threshold = 8
frequency_regularization_threshold = 6

## getting who needs regularization

In [20]:
attendance_regulariation_df['NEEDS_REGULARIZATION_FOR_WEEK'] = (attendance_regulariation_df['TOTAL_HOURS_WORKED_FOR_WEEK'] < weekly_regularization_threshold).astype(int)
attendance_regulariation_df['NEEDS_REGULARIZATION_FOR_DAY'] = (attendance_regulariation_df['AVERAGE_DAILY_HOURS_WORKED'] < daily_regularization_threshold).astype(int)
attendance_regulariation_df

,USER_EMAIL,DATE_START,TIMESHEET_D0,TIMESHEET_D1,TIMESHEET_D2,TIMESHEET_D3,TIMESHEET_D4,TIMESHEET_D5,TIMESHEET_D6,TOTAL_HOURS_WORKED_FOR_WEEK,AVERAGE_DAILY_HOURS_WORKED,NEEDS_REGULARIZATION_FOR_WEEK,NEEDS_REGULARIZATION_FOR_DAY
0,aaron25@example.com,2022-07-24,5,2,11,10,2,5,3,38,9.50,1,0
1,aaron25@example.com,2022-07-31,6,12,12,7,4,8,3,52,13.00,0,0
2,aaron25@example.com,2022-10-16,0,16,2,6,2,0,8,34,8.50,1,0
3,aaron25@example.com,2023-03-05,7,2,9,3,7,9,5,42,10.50,0,0
4,aaron25@example.com,2023-05-21,2,0,0,1,5,3,0,11,2.75,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45668,zwilliamson@example.org,2023-12-17,3,5,6,8,8,8,2,40,10.00,0,0
45669,zwilliamson@example.org,2023-12-24,4,14,4,8,4,12,12,58,14.50,0,0
45670,zwilliamson@example.org,2024-02-25,4,6,10,8,6,10,8,52,13.00,0,0
45671,zwolf@example.org,2022-06-26,0,6,0,1,2,0,0,9,2.25,1,1


## getting each users' regularization needs

In [21]:
needs_regulatization_freq = attendance_regulariation_df.groupby('USER_EMAIL')[['NEEDS_REGULARIZATION_FOR_WEEK', 'NEEDS_REGULARIZATION_FOR_DAY']].sum().reset_index()
needs_regulatization_freq['NEEDS_REGULARIZATION_FREQ'] = needs_regulatization_freq[['NEEDS_REGULARIZATION_FOR_WEEK', 'NEEDS_REGULARIZATION_FOR_DAY']].sum(axis=1)

user_weekly_regularization_flags = (attendance_regulariation_df.groupby('USER_EMAIL')['NEEDS_REGULARIZATION_FOR_WEEK'].mean() < 0.5).reset_index()
user_daily_regularization_flags = (attendance_regulariation_df.groupby('USER_EMAIL')['NEEDS_REGULARIZATION_FOR_DAY'].mean() < 0.5).reset_index()

In [22]:
needs_regulatization_freq

,USER_EMAIL,NEEDS_REGULARIZATION_FOR_WEEK,NEEDS_REGULARIZATION_FOR_DAY,NEEDS_REGULARIZATION_FREQ
0,aaron25@example.com,5,3,8
1,aaron65@example.com,3,3,6
2,aaron69@example.org,9,8,17
3,aaron85@example.net,5,3,8
4,aaronjensen@example.org,4,3,7
...,...,...,...,...
4078,ztucker@example.org,9,8,17
4079,zvaldez@example.com,3,1,4
4080,zweber@example.net,9,7,16
4081,zwilliamson@example.org,17,13,30


In [23]:
user_weekly_regularization_flags

,USER_EMAIL,NEEDS_REGULARIZATION_FOR_WEEK
0,aaron25@example.com,False
1,aaron65@example.com,False
2,aaron69@example.org,False
3,aaron85@example.net,False
4,aaronjensen@example.org,False
...,...,...
4078,ztucker@example.org,False
4079,zvaldez@example.com,False
4080,zweber@example.net,False
4081,zwilliamson@example.org,False


In [24]:
user_daily_regularization_flags

,USER_EMAIL,NEEDS_REGULARIZATION_FOR_DAY
0,aaron25@example.com,True
1,aaron65@example.com,False
2,aaron69@example.org,False
3,aaron85@example.net,True
4,aaronjensen@example.org,True
...,...,...
4078,ztucker@example.org,False
4079,zvaldez@example.com,True
4080,zweber@example.net,True
4081,zwilliamson@example.org,True


## filtering users needing regularization based on number of hours worked and average daily hours worked

In [25]:
users_needing_regularization = set(user_weekly_regularization_flags[user_weekly_regularization_flags['NEEDS_REGULARIZATION_FOR_WEEK']]['USER_EMAIL'].tolist()).intersection(user_daily_regularization_flags[user_daily_regularization_flags['NEEDS_REGULARIZATION_FOR_DAY']]['USER_EMAIL'].tolist())
users_needing_regularization

{'aboyd@example.org',
 'acoleman@example.com',
 'adam28@example.net',
 'aherrera@example.net',
 'ajacobs@example.net',
 'alanlee@example.net',
 'alara@example.com',
 'alexanderoneill@example.org',
 'alexis92@example.com',
 'alicia54@example.com',
 'alisha16@example.com',
 'allenashley@example.org',
 'allenjoseph@example.org',
 'amanda79@example.net',
 'amanda89@example.net',
 'amandabranch@example.net',
 'amandawilliams@example.net',
 'amber08@example.org',
 'amber26@example.com',
 'amcclure@example.org',
 'amiller@example.com',
 'amiller@example.org',
 'amy52@example.net',
 'amyhatfield@example.net',
 'andersonmatthew@example.com',
 'andreaoconnell@example.net',
 'andrew20@example.net',
 'andrew45@example.org',
 'andrew90@example.org',
 'andrewadams@example.org',
 'andrewayers@example.net',
 'andrewcoleman@example.org',
 'andrewhood@example.org',
 'andrewmills@example.com',
 'angela52@example.net',
 'angela69@example.org',
 'angela80@example.org',
 'angelahall@example.org',
 'annaleac

## filtering users based on frequency of regularization needed

In [26]:
needs_regulatization_freq = needs_regulatization_freq[needs_regulatization_freq['NEEDS_REGULARIZATION_FREQ'] > frequency_regularization_threshold].reset_index().drop(columns='index')
users_needing_overall_regularization = needs_regulatization_freq[needs_regulatization_freq['USER_EMAIL'].isin(users_needing_regularization)]['USER_EMAIL'].tolist()
users_needing_overall_regularization

['adam28@example.net',
 'aherrera@example.net',
 'ajacobs@example.net',
 'alanlee@example.net',
 'amber26@example.com',
 'amiller@example.com',
 'andrewayers@example.net',
 'andrewcoleman@example.org',
 'andrewmills@example.com',
 'angela69@example.org',
 'angelahall@example.org',
 'anthonyjulie@example.org',
 'ashleejones@example.org',
 'awilliams@example.net',
 'baileymichael@example.net',
 'basslisa@example.org',
 'bcarroll@example.org',
 'bennettlaura@example.net',
 'bharris@example.org',
 'brendan66@example.com',
 'brett99@example.net',
 'brewersteven@example.net',
 'brian02@example.net',
 'brianlee@example.org',
 'broberts@example.net',
 'brownmichele@example.com',
 'brucedawson@example.org',
 'bryantdarryl@example.net',
 'bryantmary@example.org',
 'bryanwilliams@example.net',
 'bsmith@example.net',
 'campbelljay@example.com',
 'camposmargaret@example.org',
 'carrolldawn@example.com',
 'carternathaniel@example.org',
 'cfoster@example.net',
 'chancaroline@example.net',
 'charlesba

## finalizing dataset

In [27]:
total_hours_worked_lifetime = attendance_regulariation_df.groupby('USER_EMAIL')['TOTAL_HOURS_WORKED_FOR_WEEK'].sum().reset_index().copy()
total_hours_worked_lifetime.rename(columns={'TOTAL_HOURS_WORKED_FOR_WEEK' : 'TOTAL_HOURS_WORKED_LIFETIME'}, inplace=True)
total_hours_worked_lifetime

,USER_EMAIL,TOTAL_HOURS_WORKED_LIFETIME
0,aaron25@example.com,231
1,aaron65@example.com,111
2,aaron69@example.org,322
3,aaron85@example.net,222
4,aaronjensen@example.org,234
...,...,...
4078,ztucker@example.org,529
4079,zvaldez@example.com,197
4080,zweber@example.net,566
4081,zwilliamson@example.org,1109


In [28]:
average_daily_hours_worked_lifetime = attendance_regulariation_df.groupby('USER_EMAIL')['AVERAGE_DAILY_HOURS_WORKED'].mean().reset_index().copy()
average_daily_hours_worked_lifetime.rename(columns={'AVERAGE_DAILY_HOURS_WORKED' : 'AVERAGE_DAILY_HOURS_WORKED_LIFETIME'}, inplace=True)
average_daily_hours_worked_lifetime

,USER_EMAIL,AVERAGE_DAILY_HOURS_WORKED_LIFETIME
0,aaron25@example.com,8.250000
1,aaron65@example.com,6.937500
2,aaron69@example.org,7.318182
3,aaron85@example.net,7.928571
4,aaronjensen@example.org,8.357143
...,...,...
4078,ztucker@example.org,8.265625
4079,zvaldez@example.com,9.850000
4080,zweber@example.net,8.323529
4081,zwilliamson@example.org,9.241667


In [29]:
final_df_ml = needs_regulatization_freq.copy()
final_df_ml = pd.merge(final_df_ml, total_hours_worked_lifetime, on='USER_EMAIL', how='inner')
final_df_ml = pd.merge(final_df_ml, average_daily_hours_worked_lifetime, on='USER_EMAIL', how='inner')
final_df_ml['NEEDS_REGULARIZATION'] = final_df_ml['USER_EMAIL'].isin(users_needing_overall_regularization).astype(int)

final_df_ml.reset_index(inplace=True)
final_df_ml.drop(columns=['index', 'USER_EMAIL'], inplace=True)
final_df_ml

,NEEDS_REGULARIZATION_FOR_WEEK,NEEDS_REGULARIZATION_FOR_DAY,NEEDS_REGULARIZATION_FREQ,TOTAL_HOURS_WORKED_LIFETIME,AVERAGE_DAILY_HOURS_WORKED_LIFETIME,NEEDS_REGULARIZATION
0,5,3,8,231,8.250000,0
1,9,8,17,322,7.318182,0
2,5,3,8,222,7.928571,0
3,4,3,7,234,8.357143,0
4,18,13,31,901,8.663462,0
...,...,...,...,...,...,...
2387,19,12,31,886,8.203704,0
2388,10,9,19,388,7.461538,0
2389,9,8,17,529,8.265625,0
2390,9,7,16,566,8.323529,0


# **ML**

## Sampling

In [59]:
over_sampling = SMOTE(sampling_strategy=0.5)
X_oversampled, y_oversampled = over_sampling.fit_resample(final_df_ml.loc[:, final_df_ml.columns != 'NEEDS_REGULARIZATION'], final_df_ml['NEEDS_REGULARIZATION'])

In [60]:
oversampled_df = pd.concat([pd.DataFrame(X_oversampled), pd.DataFrame(y_oversampled)], axis=1)
oversampled_df

,NEEDS_REGULARIZATION_FOR_WEEK,NEEDS_REGULARIZATION_FOR_DAY,NEEDS_REGULARIZATION_FREQ,TOTAL_HOURS_WORKED_LIFETIME,AVERAGE_DAILY_HOURS_WORKED_LIFETIME,NEEDS_REGULARIZATION
0,5,3,8,231,8.250000,0
1,9,8,17,322,7.318182,0
2,5,3,8,222,7.928571,0
3,4,3,7,234,8.357143,0
4,18,13,31,901,8.663462,0
...,...,...,...,...,...,...
3185,7,5,13,671,9.868372,1
3186,9,7,16,781,9.445726,1
3187,4,3,7,387,10.763407,1
3188,5,3,9,520,10.355215,1


## Train and test dataset

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X_oversampled, y_oversampled, test_size=0.2, random_state=42)

## Models

In [72]:
model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

Mean Squared Error: 0.08601043887845262
Accuracy: 0.6141549871843495


In [73]:
model = DecisionTreeRegressor(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

Mean Squared Error: 0.03134796238244514
Accuracy: 0.8593722447540116


In [74]:
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

Mean Squared Error: 0.014816927899686519
Accuracy: 0.9335308852054311


In [77]:
model = XGBRegressor(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

Mean Squared Error: 0.014796858387720574
Accuracy: 0.9336209175457224
